In [ ]:
import os
import pickle
import pandas as pd

In [ ]:
PARENT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))

with open(f'{PARENT_DIR}/models/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(url):
    df = pd.read_parquet(url)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
year = 2023
month = 3
dataset = 'yellow'

url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{dataset}_tripdata_{year}-{month:02d}.parquet'
df = read_data(url)

dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

print(f'Standard deviation for prediction {year}-{month:02d}: {y_pred.std():.4f}')